In [1]:
import numpy as np
import sys
import os

os.chdir('../')
from zodi_model.get_zmod import get_zmod
from zodi_model.solar_sp import solar_sp

# import sys
sys.path.append('/Applications/harris/idl89/lib/bridges')
from idlpy import *

new_dir = '/Users/rosaliaobrien/skysurf/github/skysurf/Zodiacal_Light_Model_IDL/'
os.chdir(new_dir)

IDL.run('.compile get_zmod')

IDL 8.9.0 (darwin x86_64 m64).
(c) 2023, L3Harris Geospatial Solutions, Inc.

Licensed for use by: Arizona State University
License: 496062
% Compiled module: ASTROLIB.
% ASTROLIB: Astronomy Library system variables have been added


'% Compiled module: STRNUMBER.\n% Compiled module: REPCHR.\n% Compiled module: REMCHAR.\n% Compiled module: GETTOK.\n% Compiled module: ZPARCHECK.\n% Compiled module: FDECOMP.\n% Compiled module: BINSRCH.\n% Compiled module: ANYWHERE.\n% Compiled module: NOWHERE.\n% Compiled module: READCOL.\n% Compiled module: CONFINT.\n% Compiled module: HONG_PHASE_FUNC.\n% Compiled module: GET_PHASE_INTEGRAL.\n% Compiled module: GET_ALBEDO.\n% Compiled module: GET_MULT.\n% Compiled module: GET_EMISS.\n% Compiled module: GET_HONG_PARAMS.\n% Compiled module: GAUSSINT.\n% Compiled module: SIMPINT.\n% Compiled module: EARTHSUN.\n% Compiled module: COLCORR.\n% Compiled module: ZPLANCK.\n% Compiled module: THERMFUNC.\n% Compiled module: PHASEFUNC.\n% Compiled module: SCATTFUNC.\n% Compiled module: ZSRCFUNC.\n% Compiled module: ZCLOUD.\n% Compiled module: MIGBAND.\n% Compiled module: SOLRING.\n% Compiled module: NEW_ISOCLOUD.\n% Compiled module: GET_PARNAMES.\n% Compiled module: GET_PARINDEX.\n% Compiled m

In [21]:
wavelength_arr = np.random.uniform(1.6,1.7,10)
day_arr = np.random.uniform(0,365,50)
lon_arr = np.random.uniform(0,360,50)
lat_arr = np.random.uniform(-90,90,50)
wavelength_arr

array([1.69381007, 1.60558003, 1.61139088, 1.64169288, 1.69099362,
       1.60993277, 1.63180426, 1.6698879 , 1.68125553, 1.64632582])

In [22]:
ratio_list = []

for wave in wavelength_arr:

    IDL.wave = wave
    IDL.day = day_arr
    IDL.lon = lon_arr
    IDL.lat = lat_arr
    
    idl_run_str = "zodi = get_zmod(wave,'skysurf',day,lon,lat)"
    IDL.run(idl_run_str)
    
    zodi_python = get_zmod(wave, 'skysurf', day_arr, lon_arr, lat_arr)
    
    ratio = IDL.zodi/zodi_python
    
    # Find indices where ratio is too high or too low
    bad_high = [i for i, r in enumerate(ratio) if r > 1.0001]
    bad_low  = [i for i, r in enumerate(ratio) if r < 0.9999]

    if bad_high or bad_low:
        print(f"\nBad wave: {wave}")

        print('\n')
        for i in bad_high:
            print((1-ratio[i])*100)
            print(day_arr[i])
            print(lon_arr[i])
            print(lat_arr[i])
            print('\n')

        print('\n')
        for i in bad_low:
            print((1-ratio[i])*100)
            print(day_arr[i])
            print(lon_arr[i])
            print(lat_arr[i])
            print('\n')

        # print('\n')


Bad wave: 1.6938100660446136


-1.1392017150556155
99.09930720617575
4.25576598727043
-2.4378447966783057


-0.03862086421453359
166.28753942168598
58.621237869282254
2.9375819025733563


-0.045734759462168206
12.03027345222866
268.1126550340103
8.710548199111386


-0.12990087244542092
39.027039873856985
339.83522839239805
5.09673326865429


-0.6094549950067751
350.0221030022607
247.81600969509128
4.445963715801199





Bad wave: 1.6055800259966533


-0.529117132785295
99.09930720617575
4.25576598727043
-2.4378447966783057


-0.014129968711285024
166.28753942168598
58.621237869282254
2.9375819025733563


-0.017073933652334006
12.03027345222866
268.1126550340103
8.710548199111386


-0.05194210429628754
39.027039873856985
339.83522839239805
5.09673326865429


-0.27289526477161097
350.0221030022607
247.81600969509128
4.445963715801199





Bad wave: 1.611390877130341


-0.5558132235127244
99.09930720617575
4.25576598727043
-2.4378447966783057


-0.01508517625394834
166.28753942168598
58.

In [4]:
ratio

array([1.        , 1.00000003, 0.99999766, 0.99999998, 0.99999995,
       1.00000031, 1.00000001, 0.9999993 , 0.99999999, 1.00000004,
       1.00000003, 0.99999999, 1.00000001, 0.99999898, 0.99999978,
       1.00000003, 1.00000001, 0.99999967, 1.00000004, 0.99999995,
       1.00000002, 1.00000002, 0.99999998, 1.00000002, 1.00000002,
       1.00000001, 1.00000002, 1.00000001, 0.99999998, 1.00000003,
       1.        , 1.00000002, 0.99999997, 1.00000002, 0.99999997,
       1.00000001, 1.00000002, 0.99999999, 1.00000004, 1.00000002,
       0.99999976, 0.99999995, 0.999999  , 1.00000155, 0.99999998,
       1.00000069, 0.99999997, 0.99999999, 1.00000001, 0.99999913])

In [5]:
IDL.lam_list

In [6]:
IDL.run('get_mult(1.7)')

'       1.0378333'

In [7]:
def get_mult(wavelength):
    """
    Get multiplicative factor for wavelengths > 1.6 microns.
    From O'Brien+2025 - exact implementation from IDL code.
    Should be = 1 for nominal HST wavelengths. When extrapolating out to 3.5 micron,
    a multiplier is added to match Kelsall better.

    Parameters:
    -----------
    wavelength : float
        Wavelength in microns

    Returns:
    --------
    float
        Multiplicative factor
    """
    if wavelength > 1.6:
        # Define wavelength and corresponding multiplier arrays
        lam_list = np.array([1.6, 2.2, 3.5, 4.9])
        mult_list = np.array([1.0, 1.227, 1.259, 1.0])

        # Interpolate between known values
        mult = np.interp(wavelength, lam_list, mult_list)
    else:
        mult = 1.0

    return mult

get_mult(1.7)

1.0378333333333334

In [8]:
np.array([1,2,3,4,5])[-2:]

array([4, 5])

In [9]:
IDL.test = np.array([1,2,3,4,5])
IDL.run('test[-2:*]')

'                     4                     5'

In [23]:
def solar_sp(wl):
    """
    Get solar spectrum estimate for any HST filter wavelength.
    
    Parameters:
    -----------
    wl : float
        Filter pivot wavelength in microns
        
    Returns:
    --------
    float
        Solar irradiance flux corresponding to the wavelength
    """
    
    # Solar spectrum flux values from Meftah+2018 processed through synphot
    flux_list = np.array([
        342116.01092185226, 3303413.8230211968, 16446170.76831905, 38540384.25308177,
        76676420.67245902, 128397575.82048588, 163383841.02144498, 188032128.2839162,
        209709058.79870653, 222817547.04015842, 231397146.5020066, 236879895.06999975,
        240377633.42061776, 240102828.21749556, 240207441.71584302, 242311737.400332,
        242466975.45810118, 239266492.1652165, 236012167.31356388, 234112458.3463153,
        232315803.03923738, 230486116.59469134, 228286915.55386606, 226470781.5675577,
        223402900.34007382, 219638777.2725546, 216293016.7440792, 211500303.47740474,
        206744275.87123755, 202388213.20880648, 194714148.80128816, 186491553.63645998,
        177695555.54650787, 169120143.4729226, 163038749.8826924, 157660709.24363747,
        152858997.50144997, 147567784.00121564, 141494800.9698657, 135587161.29024366,
        130968090.05460267, 126572711.4813484, 121667103.66006078, 117232006.96815938,
        113448941.49968067, 109595246.1106673, 105537099.95096199, 103058055.50303835,
        99379501.93762545, 96338608.89280608, 93978899.84951772, 91222947.61009558,
        88494034.69464864, 85855255.29991786, 83237630.65178712, 80917313.50268097,
        78438109.93458664, 76117181.55382372, 74046645.75447144, 71958356.82435888,
        70000915.22051653, 67960181.97848928, 65991330.90663976, 64262513.86657177,
        62599709.059710115, 61039646.51628048, 59535442.99040781, 57995180.64472865,
        56538132.5450691, 54990222.928732604, 53279644.45365265, 51993377.04684624,
        51031913.94275863, 49913322.053653, 48775552.520917535, 47584260.20074189,
        46383888.86630086, 45302398.118123665, 44374444.84588014, 43480870.197572455,
        42490931.52814285, 41500955.54348326, 40234353.02419229, 38724737.066997886,
        37359951.193162344, 36334960.86158876, 35511400.330898605, 34751678.18804755,
        34121176.04899815, 33525768.40115048, 33015642.888192605, 32516552.75856839,
        31972881.618588552, 31435922.631614495
    ])
    
    # Corresponding wavelengths in microns
    wave_list = np.array([
        0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85,
        0.9, 0.95, 1.0, 1.05, 1.1, 1.15, 1.2, 1.25, 1.3, 1.35, 1.4, 1.45, 1.5, 1.55,
        1.6, 1.65, 1.7, 1.75, 1.8, 1.85, 1.9, 1.95, 2.0, 2.05, 2.1, 2.15, 2.2, 2.25,
        2.3, 2.35, 2.4, 2.45, 2.5, 2.55, 2.6, 2.65, 2.7, 2.75, 2.8, 2.85, 2.9, 2.95,
        3.0, 3.05, 3.1, 3.15, 3.2, 3.25, 3.3, 3.35, 3.4, 3.45, 3.5, 3.55, 3.6, 3.65,
        3.7, 3.75, 3.8, 3.85, 3.9, 3.95, 4.0, 4.05, 4.1, 4.15, 4.2, 4.25, 4.3, 4.35,
        4.4, 4.45, 4.5, 4.55, 4.6, 4.65, 4.7, 4.75, 4.8, 4.85
    ])
    
    # Find closest wavelength
    diff_arr = np.abs(wave_list - wl)
    idx = np.argmin(diff_arr)
    # print(idx)
    return flux_list[idx]

python_sol_val = solar_sp(1.69381)

In [24]:
IDL.run('sol_val = solar_sp(1.69381)')

In [25]:
python_sol_val/IDL.sol_val

1.0000000246581375